In [202]:
import numpy as np
import matplotlib.pyplot as plt

In [203]:
# Change the following to match the simulation data

folder_path = 'E:/fusion_forces/fixed_rad/13aug24/'

nrod = 3

if nrod == 9:
    # 9 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[114,172,802,843,894]]
    t_hem_list = [[10,111,24,85,1064]]
    t_ihem_list = [[10,111,24,85,1064]] # irreversible hemifusion if that exists or longest live stalk start if not (if snares traverse cut off there); 0 if no stalk occurs
    t_fus_list = [[48,230,95,105,1110]] # fusion time or end of longest live stalk if no fusion; 0 if no stalk occurs
elif nrod == 7:
    # 7 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[252,612,824,863,952]]
    t_hem_list = [[139,37,1195,121,257]]
    t_ihem_list = [[801,37,1195,121,257]]
    t_fus_list = [[833,130,1213,283,435]]
elif nrod == 6:
    # 6 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[108,184,330,758,943]]
    t_hem_list = [[1499,386,168,187,1463]]
    t_ihem_list = [[0,386,168,187,1463]]
    t_fus_list = [[0,505,341,342,1467]]
elif nrod == 5:
    # 5 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[25,408,535,564,597]]
    t_hem_list = [[871,792,250,284,178]]
    t_ihem_list = [[871,1333,250,444,704]]
    t_fus_list = [[927,1487,375,496,1120]]
elif nrod == 4:
    # 4 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[99,428,686,832,954]]
    t_hem_list = [[1062,1115,521,664,895]]
    t_ihem_list = [[1316,1115,521,913,895]]
    t_fus_list = [[1499,1147,566,963,897]]
elif nrod == 3:
    # 3 SNARE data
    rtmd_arr = [5.91]
    delta_h = [1.50]
    run_list = [[5,5,5,5,5]]
    seed_list = [[113,454,569,601,833]]
    t_hem_list = [[215,1499,1499,1499,445]]
    t_ihem_list = [[215,0,0,0,445]]
    t_fus_list = [[253,0,0,0,447]]

In [204]:
dt = 0.068/1000 # timestep in microseconds
frame_dt = 20000 # in steps
check_period = 200 # in steps

# Function Definitions

In [205]:
def get_tmd_pos(run, seed, t_start, t_stop):
    
    linker_up_dir = np.load(path_prefix+'run_%d_dir_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_dir = np.load(path_prefix+'run_%d_dir_dn_%d.npy'%(run, seed), allow_pickle=True)
    
    linker_up_pos = np.load(path_prefix+'run_%d_ct_up_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_dn_pos = np.load(path_prefix+'run_%d_ct_dn_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma

    tmd_up_pos = - linker_up_dir + linker_up_pos
    tmd_dn_pos = - linker_dn_dir + linker_dn_pos

    tmd_up_pos = tmd_up_pos[t_start:t_stop, ...]
    tmd_dn_pos = tmd_dn_pos[t_start:t_stop, ...]
    
    tmd_up_pos = tmd_up_pos*0.88
    tmd_dn_pos = tmd_dn_pos*0.88 # in nm

    return tmd_up_pos, tmd_dn_pos

In [206]:
def get_linker_force_cart(run, seed, t_start, t_stop):
   
    # Load linker forces
    linker_up_f_comp = np.load(path_prefix+'run_%d_force_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_f_comp = np.load(path_prefix+'run_%d_force_dn_%d.npy'%(run, seed), allow_pickle=True)
    linker_up_f_comp = linker_up_f_comp[t_start:t_stop, ...]
    linker_dn_f_comp = linker_dn_f_comp[t_start:t_stop, ...]
    linker_up_f_comp = -linker_up_f_comp*0.6*4.1/0.88 # in pN
    linker_dn_f_comp = -linker_dn_f_comp*0.6*4.1/0.88 # in pN
    linker_up_f = np.linalg.norm(linker_up_f_comp, axis=2)
    linker_dn_f = np.linalg.norm(linker_dn_f_comp, axis=2)
    
    return linker_up_f, linker_dn_f, linker_up_f_comp, linker_dn_f_comp # in pN

In [207]:
def get_snare_force_tot(run, seed, t_start, t_stop):
    
    # Load SNARE COM forces (including linker forces)
    snare_data = np.load(path_prefix+'run_%d_snare_force_%d.npy'%(run, seed), allow_pickle=True)
    snare_data = snare_data[t_start:t_stop, ...]
    # Convert SNARE COM forces to pN
    snare_data = snare_data*0.6*4.1/0.88
    
    return snare_data # in pN

In [208]:
def get_snare_entropic_force(snare_data, linker_up_f_comp, linker_dn_f_comp):
    snare_f = snare_data[:] - linker_up_f_comp[:] - linker_dn_f_comp[:] # Compute total force on the SNARE, excluding linker forces (in pN)
    return snare_f

In [209]:
def get_ring_center(snare_ctd_pos):
    # Find the centroid of the SNARE CTDs
    snare_ctd_centroid = np.mean(snare_ctd_pos, axis=1)
    return snare_ctd_centroid

In [210]:
def get_cterm_pos(run, seed, t_start, t_stop):
    
    # Load linker positions for all other snares
    linker_up_pos = np.load(path_prefix+'run_%d_ct_up_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_dn_pos = np.load(path_prefix+'run_%d_ct_dn_%d.npy'%(run, seed), allow_pickle=True) # in units of sigma
    linker_up_pos = linker_up_pos[t_start:t_stop, ...]
    linker_dn_pos = linker_dn_pos[t_start:t_stop, ...]

    # Find the mean position of the CTD for each snare
    snare_ctd_pos = (linker_up_pos + linker_dn_pos)/2 * 0.88 # in nm
    snare_ctd_pos_avg = np.mean(snare_ctd_pos, axis=0) # in nm
    
    return snare_ctd_pos, snare_ctd_pos_avg

In [211]:
def get_ring_radius(snare_ctd_pos, snare_ctd_centroid):
    # Compute the distance of each SNARE CTD from the centroid
    snare_r_centroid = snare_ctd_pos - snare_ctd_centroid[:, np.newaxis, :]
    # Compute the magnitude of the distance of each SNARE CTD from the centroid (in nm)
    snare_r_centroid_magn = np.sqrt(snare_r_centroid[..., 0]**2 + snare_r_centroid[..., 1]**2) # in units of sigma
    snare_r_centroid_magn = snare_r_centroid_magn*0.88 # in nm
    # compute the mean ring radius
    #snare_r_centroid_magn_mean = np.mean(snare_r_centroid_magn, axis=0)
    #snare_r_centroid_magn_std = np.std(snare_r_centroid_magn, axis=0)
    #snare_r_maxmin = np.max(snare_r_centroid_magn, axis=0)/np.min(snare_r_centroid_magn, axis=0)
    #return snare_r_centroid_magn_mean, snare_r_centroid_magn_std, snare_r_maxmin
    return snare_r_centroid_magn

In [212]:
def get_linker_norm(run, seed, t_start, t_stop):
    
    # Load linker direction vectors (in units of sigma)
    linker_up_dir = np.load(path_prefix+'run_%d_dir_up_%d.npy'%(run, seed), allow_pickle=True)
    linker_dn_dir = np.load(path_prefix+'run_%d_dir_dn_%d.npy'%(run, seed), allow_pickle=True)
    linker_up_dir = linker_up_dir[t_start:t_stop, ...]
    linker_dn_dir = linker_dn_dir[t_start:t_stop, ...]
    #print(linker_up_dir.shape)
    #print(linker_dn_dir.shape)
    
    # Compute the linker vector magnitudes (in units of sigma)
    linker_up_magn = np.linalg.norm(linker_up_dir, axis=2, keepdims=True)
    linker_dn_magn = np.linalg.norm(linker_dn_dir, axis=2, keepdims=True)
   
    # Check that both linker_up_dir and linker_dn_dir are pointing from CTD to TMD
    # z_TMD - z_CTD > 0 for the up linkers, so linker_up_dir[2] should be positive
    # z_TMD - z_CTD < 0 for the dn linkers, so linker_dn_dir[2] should be negative
    if np.mean(linker_up_dir[:10,:,2]) < 0: 
        #print('Inverting linker_up_dir')
        linker_up_dir = -linker_up_dir
    if np.mean(linker_dn_dir[:10,:,2]) > 0: 
        #print('Inverting linker_dn_dir')
        linker_dn_dir = -linker_dn_dir

    # Compute the linker unit vectors (pointing from CTD to TMD)
    linker_up_dir_norm = linker_up_dir / (linker_up_magn + 1e-10) # add 1e-10 to avoid division by zero
    linker_dn_dir_norm = linker_dn_dir / (linker_dn_magn + 1e-10) # add 1e-10 to avoid division by zero

    return linker_up_dir_norm, linker_dn_dir_norm

In [213]:
def get_polar_dir(tmd_pos_avg, ori):

    ori = get_ring_center(tmd_pos_avg)
    
    # Translate linker COM positions relative to the new origin
    tmd_pos_avg = tmd_pos_avg - ori[:, None, :]
    
    # Get unit vectors for the r and theta directions for each SNARE
    rx = tmd_pos_avg[..., 0]/np.sqrt(tmd_pos_avg[..., 0]**2 + tmd_pos_avg[..., 1]**2)
    ry = tmd_pos_avg[..., 1]/np.sqrt(tmd_pos_avg[..., 0]**2 + tmd_pos_avg[..., 1]**2)

    thetax = -ry
    thetay = rx

    return rx, ry, thetax, thetay

In [214]:
def get_polar_force(snare_f_x, snare_f_y, snare_f_z, rx, ry, thetax, thetay):
    
    # Compute the force components in the r and theta directions
    snare_f_r = snare_f_x*rx + snare_f_y*ry
    snare_f_theta = snare_f_x*thetax + snare_f_y*thetay
    return snare_f_r, snare_f_theta, snare_f_z

In [215]:
def get_polar_pos(snare_ctd_pos, rx, ry, thetax, thetay):
    
    # Get unit vectors for the r and theta directions for each SNARE
    r = snare_ctd_pos[..., 0]*rx + snare_ctd_pos[..., 1]*ry
    theta = snare_ctd_pos[..., 0]*thetax + snare_ctd_pos[..., 1]*thetay

    return r, theta

In [216]:
from scipy.signal import correlate

def autocorr(x):
    
    #Normalize the result to have it range from -1 to 1, so the autocorrelation at lag 0 is exactly 1
    norm = np.sum(x ** 2)
    
    # The 'same' mode returns the central part of the correlation
    # that is the same size as the input signal.
    acf = correlate(x, x, mode='same') / norm
    
    # Since the result includes negative lags, we only want the second half
    # which corresponds to the positive lags (including lag 0 at the center)
    acf = acf[acf.size // 2:]
    return acf

# Calculations

In [217]:
for index in range(len(rtmd_arr)):
    
    path_prefix = folder_path + 'ves_ves_rtmd_ini_%.2f_deltah_%.2f_Nunzip_10.00_lp_0.50_nrod_%d_rod_r_0.38_dves_55.9_tension_0.05_'%(rtmd_arr[index],delta_h[index],nrod)
    
    run_arr = run_list[index]
    seed_arr = seed_list[index]
    t_hem_arr = t_hem_list[index]
    t_ihem_arr = t_ihem_list[index]
    t_fus_arr = t_fus_list[index]

    snare_f_x_arr_all = []
    snare_f_y_arr_all = []
    snare_f_z_arr_all = []
    snare_data_x_arr_all = []
    snare_data_y_arr_all = []
    snare_data_z_arr_all = []
    linker_up_f_x_arr_all = []
    linker_up_f_y_arr_all = []
    linker_up_f_z_arr_all = []
    linker_dn_f_x_arr_all = []
    linker_dn_f_y_arr_all = []
    linker_dn_f_z_arr_all = []
    linker_up_arr_all = []
    linker_dn_arr_all = []
    snare_ctd_pos_arr_all = []
    snare_ctd_pos_avg_arr = []
    tmd_pos_up_arr = []
    tmd_pos_dn_arr = []

    ring_radius_arr = []
    ring_radius_std_arr = []
    #ring_radius_maxmin_arr = []
    n_measurements_arr = []

    for i in range(len(run_arr)):
        run = run_arr[i]
        seed = seed_arr[i]
        print('run = %d, seed = %d'%(run, seed))

        frame_start = t_ihem_arr[i]
        frame_stop = t_fus_arr[i]-1
        if frame_stop > 0:
            t_start = int((frame_start)*frame_dt/check_period)
            t_stop = int((frame_stop)*frame_dt/check_period)

            linker_up_f_all, linker_dn_f_all, linker_up_f_comp_all, linker_dn_f_comp_all = get_linker_force_cart(run, seed, t_start, t_stop)
            snare_data_all = get_snare_force_tot(run, seed, t_start, t_stop)
            snare_f_all = get_snare_entropic_force(snare_data_all, linker_up_f_comp_all, linker_dn_f_comp_all)
            snare_ctd_pos, snare_ctd_pos_avg = get_cterm_pos(run, seed, t_start, t_stop)
            tmd_pos_up, tmd_pos_dn = get_tmd_pos(run, seed, t_start, t_stop)
            tmd_pos_avg = (tmd_pos_up + tmd_pos_dn)/2
            #ring_radius, ring_radius_std, ring_radius_maxmin = get_ring_radius(snare_ctd_pos, get_ring_center(snare_ctd_pos))
            ring_radius = get_ring_radius(tmd_pos_avg, get_ring_center(tmd_pos_avg))

            n_measurements = np.shape(snare_data_all)[0]
            n_measurements_arr.append(n_measurements)

            snare_f_x_arr_all.append(snare_f_all[..., 0])
            snare_f_y_arr_all.append(snare_f_all[..., 1])
            snare_f_z_arr_all.append(snare_f_all[..., 2])
            snare_data_x_arr_all.append(snare_data_all[..., 0])
            snare_data_y_arr_all.append(snare_data_all[..., 1])
            snare_data_z_arr_all.append(snare_data_all[..., 2])
            linker_up_f_x_arr_all.append(linker_up_f_comp_all[..., 0])
            linker_up_f_y_arr_all.append(linker_up_f_comp_all[..., 1])
            linker_up_f_z_arr_all.append(linker_up_f_comp_all[..., 2])
            linker_dn_f_x_arr_all.append(linker_dn_f_comp_all[..., 0])
            linker_dn_f_y_arr_all.append(linker_dn_f_comp_all[..., 1])
            linker_dn_f_z_arr_all.append(linker_dn_f_comp_all[..., 2])
            linker_up_arr_all.append(linker_up_f_all)
            linker_dn_arr_all.append(linker_dn_f_all)
            snare_ctd_pos_arr_all.append(snare_ctd_pos)
            snare_ctd_pos_avg_arr.append(snare_ctd_pos_avg)
            tmd_pos_up_arr.append(tmd_pos_up)
            tmd_pos_dn_arr.append(tmd_pos_dn)

            ring_radius_arr.append(ring_radius)
            #ring_radius_std_arr.append(ring_radius_std)
            #ring_radius_maxmin_arr.append(ring_radius_maxmin)
            
            del tmd_pos_up, tmd_pos_dn, tmd_pos_avg, snare_ctd_pos, snare_ctd_pos_avg, ring_radius
            del linker_up_f_comp_all, linker_dn_f_comp_all, snare_data_all, snare_f_all, linker_up_f_all, linker_dn_f_all
            #del ring_radius, ring_radius_std, ring_radius_maxmin


    # Flatten all force arrays into 2D arrays of shape (n_measurements, nsnare) 

    snare_f_x_arr_all = np.concatenate(snare_f_x_arr_all, axis=0)
    snare_f_y_arr_all = np.concatenate(snare_f_y_arr_all, axis=0)
    snare_f_z_arr_all = np.concatenate(snare_f_z_arr_all, axis=0)
    snare_data_x_arr_all = np.concatenate(snare_data_x_arr_all, axis=0)
    snare_data_y_arr_all = np.concatenate(snare_data_y_arr_all, axis=0)
    snare_data_z_arr_all = np.concatenate(snare_data_z_arr_all, axis=0)
    linker_up_f_x_arr_all = np.concatenate(linker_up_f_x_arr_all, axis=0)
    linker_up_f_y_arr_all = np.concatenate(linker_up_f_y_arr_all, axis=0)
    linker_up_f_z_arr_all = np.concatenate(linker_up_f_z_arr_all, axis=0)
    linker_dn_f_x_arr_all = np.concatenate(linker_dn_f_x_arr_all, axis=0)
    linker_dn_f_y_arr_all = np.concatenate(linker_dn_f_y_arr_all, axis=0)
    linker_dn_f_z_arr_all = np.concatenate(linker_dn_f_z_arr_all, axis=0)
    linker_up_arr_all = np.concatenate(linker_up_arr_all, axis=0)
    linker_dn_arr_all = np.concatenate(linker_dn_arr_all, axis=0)
    snare_ctd_pos_arr_all = np.concatenate(snare_ctd_pos_arr_all, axis=0)
    tmd_pos_up_arr = np.concatenate(tmd_pos_up_arr, axis=0)
    tmd_pos_dn_arr = np.concatenate(tmd_pos_dn_arr, axis=0)
    ring_radius_arr = np.concatenate(ring_radius_arr, axis=0)

    # Compute total linker forces (sum of up and down linkers)
    linker_f_x_arr_all = linker_up_f_x_arr_all + linker_dn_f_x_arr_all
    linker_f_y_arr_all = linker_up_f_y_arr_all + linker_dn_f_y_arr_all
    linker_f_z_arr_all = linker_up_f_z_arr_all + linker_dn_f_z_arr_all

    # Compute averages and standard deviations ISNARE

    corr = 1
    n_measurements = np.sum(n_measurements_arr) # total number of measurements
    #print('Number of measurements per SNARE = %d'%n_measurements)
    #print('Measurement time: %.2f us'%(n_measurements*dt*check_period))

    # Calculate the linker force magnitude in the xy plane along the mean direction
    linker_f_xy_magn_avg = []
    for i in range(np.shape(linker_f_x_arr_all)[1]):
        linker_f_xy_magn_avg.append(np.sqrt(np.mean(linker_f_x_arr_all[:, i])**2 + np.mean(linker_f_y_arr_all[:, i])**2))
    #print('Linker force magnitude in the mean direction in xy plane')
    #print('Mean (pN): ',linker_f_xy_magn_avg)

    # Get the unit vectors of the linker forces in the radial and theta directions
    '''
    ld_dir_rad_x = -np.mean(linker_f_x_arr_all, axis = 0)/linker_f_xy_magn_avg
    ld_dir_rad_y = -np.mean(linker_f_y_arr_all, axis = 0)/linker_f_xy_magn_avg
    ld_dir_rad_magn = np.sqrt(ld_dir_rad_x**2 + ld_dir_rad_y**2)
    print('Linker direction in the radial direction')
    print('Mean (pN): ', ld_dir_rad_magn)
    ld_dir_theta_x = -ld_dir_rad_y
    ld_dir_theta_y = ld_dir_rad_x
    '''

    # Find the mean position of each SNARE's TMD in cartesian coords
    tmd_pos_avg_arr = (tmd_pos_up_arr + tmd_pos_dn_arr)/2
    tmd_pos_avg = np.mean(tmd_pos_avg_arr, axis=0)

    # Get radial and theta unit vectors pointing from the ring center to the TMDs
    ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y = get_polar_dir(tmd_pos_avg_arr, get_ring_center(tmd_pos_avg_arr))

    # Convert all forces to polar coordinates
    snare_data_rad_arr_all, snare_data_theta_arr_all, snare_data_z_arr_all = get_polar_force(snare_data_x_arr_all, snare_data_y_arr_all, snare_data_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
    snare_f_rad_arr_all, snare_f_theta_arr_all, snare_f_z_arr_all = get_polar_force(snare_f_x_arr_all, snare_f_y_arr_all, snare_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
    linker_up_f_rad_arr_all, linker_up_f_theta_arr_all, linker_up_f_z_arr_all = get_polar_force(linker_up_f_x_arr_all, linker_up_f_y_arr_all, linker_up_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)
    linker_dn_f_rad_arr_all, linker_dn_f_theta_arr_all, linker_dn_f_z_arr_all = get_polar_force(linker_dn_f_x_arr_all, linker_dn_f_y_arr_all, linker_dn_f_z_arr_all, ld_dir_rad_x, ld_dir_rad_y, ld_dir_theta_x, ld_dir_theta_y)

    # Compute total linker forces (sum of up and down linkers)
    linker_f_rad_arr_all = linker_up_f_rad_arr_all + linker_dn_f_rad_arr_all
    linker_f_theta_arr_all = linker_up_f_theta_arr_all + linker_dn_f_theta_arr_all
    linker_f_z_arr_all = linker_up_f_z_arr_all + linker_dn_f_z_arr_all

    # Compute entropic tension
    ring_radius_arr_avg_inst = np.mean(ring_radius_arr, axis = 1)
    gamma_ent_arr = np.sum(snare_f_rad_arr_all, axis = 1)/(2*np.pi*(np.mean(ring_radius_arr, axis = 1)))

    # Prep data to save
    ring_rad_avg = np.mean(ring_radius_arr_avg_inst)
    ring_rad_std = np.std(ring_radius_arr_avg_inst)

    gamma_ent_avg = np.mean(gamma_ent_arr)
    gamma_ent_std = np.std(gamma_ent_arr)

    T_zip_avg = np.mean((linker_up_arr_all+linker_dn_arr_all)/2) 
    T_zip_std = np.std((linker_up_arr_all+linker_dn_arr_all)/2)

    F_squeeze_up = np.sum(linker_up_f_z_arr_all, axis = 1)
    F_squeeze_dn = np.sum(linker_dn_f_z_arr_all, axis = 1)
    F_squeeze_avg = np.mean((F_squeeze_up - F_squeeze_dn)/2)
    F_squeeze_std = np.std((F_squeeze_up - F_squeeze_dn)/2)

    f_squeeze_avg = np.mean((linker_up_f_z_arr_all - linker_dn_f_z_arr_all)/2)
    f_squeeze_std = np.std((linker_up_f_z_arr_all - linker_dn_f_z_arr_all)/2)

    T_rad_tot_avg = np.mean(linker_f_rad_arr_all)
    T_rad_tot_std = np.std(linker_f_rad_arr_all)
    T_theta_tot_avg = np.mean(linker_f_theta_arr_all)
    T_theta_tot_std = np.std(linker_f_theta_arr_all)

    fent_rad_avg = np.mean(snare_f_rad_arr_all)
    fent_rad_std = np.std(snare_f_rad_arr_all)
    fent_theta_avg = np.mean(snare_f_theta_arr_all)
    fent_theta_std = np.std(snare_f_theta_arr_all)
    fent_z_avg = np.mean(snare_f_z_arr_all)
    fent_z_std = np.std(snare_f_z_arr_all)

    # Save the data to a .txt file
    fname = 'force_data_stalk_nrod_%d_rtmd_ini_%.2f.txt'%(nrod, rtmd_arr[index])

    with open(fname, 'w') as file:
        file.write("Number of SNAREs\n")
        file.write(f"{nrod}\n")
        file.write("TMD ring constraint (nm)\n")
        file.write(f"{rtmd_arr[index]*0.88}\n")
        file.write("Number of measurements per SNARE\n")
        file.write(f"{n_measurements}\n")
        file.write("TMD ring radius (nm)\n")
        file.write(f"{ring_rad_avg}\n")
        file.write(f"{ring_rad_std}\n")
        file.write("Zippering force per LD (pN)\n")
        file.write(f"{T_zip_avg}\n")
        file.write(f"{T_zip_std}\n")
        file.write("Total squeezing force (pN)\n")
        file.write(f"{F_squeeze_avg}\n")
        file.write(f"{F_squeeze_std}\n")
        file.write("Squeezing force per LD (pN)\n")
        file.write(f"{f_squeeze_avg}\n")
        file.write(f"{f_squeeze_std}\n")
        file.write("Radial linker force (both LDs) (pN)\n")
        file.write(f"{T_rad_tot_avg}\n")
        file.write(f"{T_rad_tot_std}\n")
        file.write("Theta linker force (both LDs) (pN)\n")
        file.write(f"{T_theta_tot_avg}\n")
        file.write(f"{T_theta_tot_std}\n")
        file.write("Radial entropic force (pN)\n")
        file.write(f"{fent_rad_avg}\n")
        file.write(f"{fent_rad_std}\n")
        file.write("Theta entropic force (pN)\n")
        file.write(f"{fent_theta_avg}\n")
        file.write(f"{fent_theta_std}\n")
        file.write("Z entropic force (pN)\n")
        file.write(f"{fent_z_avg}\n")
        file.write(f"{fent_z_std}\n")
        file.write("Entropic tension (pN/nm)\n")
        file.write(f"{gamma_ent_avg}\n")
        file.write(f"{gamma_ent_std}\n")


run = 5, seed = 113
run = 5, seed = 454
run = 5, seed = 569
run = 5, seed = 601
run = 5, seed = 833
